# **Pipeline Mixture of Experts**

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## **1.Data preparation**
### ELIA + MTO

In [ ]:
# Retrieve data
df_elia = pd.read_csv("../../Data/ELIA.csv", sep=";")
df_mto = pd.read_csv("../../Data/MTO_BELGIQUE_18_to_25.csv", sep=",")


df_offshore = df_elia[df_elia["Offshore/onshore"] == "Offshore"].copy()         # Keep only Offshore rows
df_offshore["Datetime"] = pd.to_datetime(df_offshore["Datetime"], utc=True)     # Convert Datetime to actual datetime
df_offshore = df_offshore.dropna(how="any")                                     # Drop empty rows
df_offshore = df_offshore.sort_values("Datetime").reset_index(drop=True)        # Sort values in chronological order

df_mto["valid_time"] = pd.to_datetime(df_mto["valid_time"], utc=True)           # Convert Datetime to actual datetime
df_mto = df_mto.dropna(how="any")                                               # Drop empty rows
df_mto = df_mto.sort_values("valid_time").reset_index(drop=True)                # Sort values in chronological order

# Visualization options
#df_elia.head(5)
#df_offshore.head(3)
#df_mto.head()

print("Offshore data")
print("=================================================")
print("Oldest date :", df_offshore["Datetime"].min())
print("Newest date :", df_offshore["Datetime"].max())
print("")
print("MTO data")
print("=================================================")
print("Oldest date :", df_mto["valid_time"].min())
print("Newest date :", df_mto["valid_time"].max())

Offshore data
Oldest date : 2016-01-12 23:00:00+00:00
Newest date : 2025-12-01 22:45:00+00:00

MTO data
Oldest date : 2018-01-01 00:00:00+00:00
Newest date : 2025-11-23 23:00:00+00:00


### Merge ELIA + MTO

In [ ]:
# Convert ELIA to hourly resolution
df_offshore_hourly_exact = df_offshore[df_offshore["Datetime"].dt.minute == 0].copy()
df_offshore_hourly_exact.head()

df_merged = df_offshore_hourly_exact.merge(
    df_mto,
    left_on="Datetime",
    right_on="valid_time",
    how="inner"
)
df_merged.head()

print("Merged data")
print("=================================================")
print("Oldest date :", df_merged["Datetime"].min())
print("Newest date :", df_merged["Datetime"].max())

,Datetime,Resolution code,Offshore/onshore,Region,Grid connection type,Measured & Upscaled,Most recent forecast,Most recent P10,Most recent P90,Day Ahead 11AM forecast,...,u100,v100,u10,v10,t2m,msl,sst,sp,latitude,longitude
0,2018-01-01 00:00:00+00:00,PT15M,Offshore,Federal,Elia,821.80,831.49,797.38,865.06,833.00,...,17.891388,12.289810,13.044540,9.531647,281.30188,99547.310,280.7588,99568.836,51.5,3.0
1,2018-01-01 01:00:00+00:00,PT15M,Offshore,Federal,Elia,812.10,832.30,793.81,869.35,833.10,...,19.008880,7.765930,14.131149,6.141098,281.37300,99638.310,280.7588,99659.280,51.5,3.0
2,2018-01-01 02:00:00+00:00,PT15M,Offshore,Federal,Elia,812.10,834.23,796.77,869.35,833.73,...,17.849350,5.749756,13.414413,4.661072,281.32764,99812.190,280.7588,99833.780,51.5,3.0
3,2018-01-01 03:00:00+00:00,PT15M,Offshore,Federal,Elia,809.06,830.88,787.94,869.17,832.64,...,16.603897,4.897415,12.725754,3.991196,281.15686,99922.060,280.7588,99943.300,51.5,3.0
4,2018-01-01 04:00:00+00:00,PT15M,Offshore,Federal,Elia,802.93,827.05,775.09,870.18,829.37,...,14.488632,5.378235,11.158417,4.428650,280.96695,99985.375,280.7588,100007.720,51.5,3.0


### Weather data

In [ ]:
# Retrieve data
df_mto = pd.read_csv("../../Data/MTO_BELGIQUE_18_to_25.csv", sep=",")
print("Oldest date :", df_mto["valid_time"].min())
print("Newest date :", df_mto["valid_time"].max())
df_mto.head(50)

Oldest date : 2018-01-01 00:00:00
Newest date : 2025-11-23 23:00:00


,valid_time,u100,v100,u10,v10,t2m,msl,sst,sp,latitude,longitude
0,2018-01-01 00:00:00,17.891388,12.289810,13.044540,9.531647,281.30188,99547.310,280.75880,99568.836,51.5,3.0
1,2018-01-01 01:00:00,19.008880,7.765930,14.131149,6.141098,281.37300,99638.310,280.75880,99659.280,51.5,3.0
2,2018-01-01 02:00:00,17.849350,5.749756,13.414413,4.661072,281.32764,99812.190,280.75880,99833.780,51.5,3.0
3,2018-01-01 03:00:00,16.603897,4.897415,12.725754,3.991196,281.15686,99922.060,280.75880,99943.300,51.5,3.0
4,2018-01-01 04:00:00,14.488632,5.378235,11.158417,4.428650,280.96695,99985.375,280.75880,100007.720,51.5,3.0
5,2018-01-01 05:00:00,13.108093,6.551071,10.175552,5.376297,280.83783,100030.000,280.75880,100051.750,51.5,3.0
6,2018-01-01 06:00:00,12.092590,8.058746,9.458969,6.582321,280.70764,100030.690,280.75880,100052.195,51.5,3.0
7,2018-01-01 07:00:00,11.249008,9.396912,8.729462,7.603607,280.65350,100036.940,280.75880,100058.625,51.5,3.0
8,2018-01-01 08:00:00,9.557251,10.433273,7.133972,8.263519,280.61160,100010.310,280.75880,100031.360,51.5,3.0
9,2018-01-01 09:00:00,7.701599,11.477905,5.403732,8.818115,280.54782,99980.375,280.75880,100001.560,51.5,3.0
